# Company Metrics #
#### Financial ####
- Realized Revenue
- Realized ARR
- ARPU
- Distribution of Pricing Tiers

#### Operations ####
- Order Volume
- Order Volume By Brand
- Average Fulfillment Time Per Order (AFT)
- Fulfillment Error Rate (FER)
- Mispick Rate
- Missing Orders Rate 

#### Product ####
- Mobile App Error Rate
- Landing Page Visitors (monthly)
- Landing Page Signup Conversion Rate

In [16]:
# Reminder -- don't forget to set up your connection through the terminal
# cloud-sql-proxy --port 5432 --auto-iam-authn packsmith:us-west1:postgres-1

import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact

load_dotenv()

def connect_to_postgres():
    host = os.getenv('DB_HOST')
    port = os.getenv('DB_PORT')
    database = os.getenv('DB_NAME')
    user = os.getenv('DB_USER')
    password = os.getenv('DB_PASSWORD')
    
    connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_string)
    connection = engine.connect()
    print("Connection established successfully.")
    return connection

from db_connection import connect_to_postgres

# Boot up the Postgres connection
connection = connect_to_postgres()


Connection established successfully.


# Financial Metrics

In [17]:
# Load up revenue data
revenue_query = """
SELECT *
FROM billing_records
"""

In [39]:
finance_query = """
SELECT 
    DATE_TRUNC('week', created_at) AS week,
    DATE_TRUNC('month', created_at) AS month,
    CASE 
        WHEN (f.destination::json->>'country') = 'Australia' THEN 'Australia'
        WHEN (f.destination::json->>'country') = 'United States' THEN 'United States'
        ELSE 'Other'
    END AS country,
    f.type,
    COUNT(id) AS fulfillment_count
FROM fulfillments f
WHERE f.deleted_at IS NULL
AND f.status = 'fulfilled'
AND f.created_at >= '2024-01-01 00:00:00+00'
GROUP BY week, month, country, f.type
ORDER BY week, month, country
"""

# Execute query and load the data into a DataFrame
df = pd.read_sql(finance_query, connection)

# Convert 'week' and 'month' to datetime and format as MM-DD
df['week'] = pd.to_datetime(df['week'])
df['month'] = pd.to_datetime(df['month'])
df['calendar_week'] = df['week'].dt.strftime('%Y-%m-%d')
df['calendar_month'] = df['month'].dt.strftime('%Y-%m')

# Function to display the data and graph
def display_orders(view_by, order_type, country):
    # Filter data based on order type
    if order_type == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['type'] == order_type.lower()]

    # Pivot the data based on the selected view (week or month)
    if view_by == 'Week':
        if country == 'All':
            pivot_df = filtered_df.pivot_table(index='calendar_week', columns='country', values='fulfillment_count', aggfunc='sum').fillna(0).astype(int)
            pivot_df['All countries'] = pivot_df.sum(axis=1)
            pivot_df = pivot_df[['All countries'] + [col for col in pivot_df.columns if col != 'All countries']]
        else:
            pivot_df = filtered_df[filtered_df['country'] == country].pivot_table(index='calendar_week', columns='country', values='fulfillment_count', aggfunc='sum').fillna(0).astype(int)
    else:
        if country == 'All':
            pivot_df = filtered_df.pivot_table(index='calendar_month', columns='country', values='fulfillment_count', aggfunc='sum').fillna(0).astype(int)
            pivot_df['All countries'] = pivot_df.sum(axis=1)
            pivot_df = pivot_df[['All countries'] + [col for col in pivot_df.columns if col != 'All countries']]
        else:
            pivot_df = filtered_df[filtered_df['country'] == country].pivot_table(index='calendar_month', columns='country', values='fulfillment_count', aggfunc='sum').fillna(0).astype(int)

    # Reset the index for graphing purposes
    graph_df = pivot_df.reset_index().melt(id_vars=[pivot_df.index.name], var_name='country', value_name='fulfillment_count')

    # Plot the graph
    fig = px.bar(graph_df, x=pivot_df.index.name, y='fulfillment_count', color='country', barmode='stack', title=f"Fulfilled Orders By {view_by}")
    fig.update_xaxes(title_text=f"{view_by}", tickformat='%Y-%m-%d' if view_by == 'Week' else '%Y-%m')
    fig.update_yaxes(title_text='Fulfillment Count')
    fig.show()

    # Display the pivot table
    display(Markdown(f"## Fulfilled Orders By {view_by}"))
    display(pivot_df)

# Create dropdown widgets for view by, order type, and country
view_by_dropdown = widgets.Dropdown(
    options=['Week', 'Month'],
    value='Week',
    description='View By:'
)

order_type_dropdown = widgets.Dropdown(
    options=['All', 'Retail', 'Wholesale'],
    value='All',
    description='Order Type:'
)

country_dropdown = widgets.Dropdown(
    options=['All', 'Australia', 'United States'],
    value='All',
    description='Country:'
)

# Use interact to create the dynamic output based on the selected filters
interact(display_orders, view_by=view_by_dropdown, order_type=order_type_dropdown, country=country_dropdown)


interactive(children=(Dropdown(description='View By:', options=('Week', 'Month'), value='Week'), Dropdown(desc…

<function __main__.display_orders(view_by, order_type, country)>

# Operations

In [19]:
# Load up the Ops data
query = """
SELECT
    m."name" AS merchant,
    f."name" AS "order",
    f."type" AS order_type,
    f.requested_at AS order_requested,
    f.created_at AS order_received,
    fj.created_at AS order_accepted,
    s.created_at AS order_fulfilled,
    s.delivered_at AS order_delivered,
    sup.address->>'country' AS from_country,
    sup.address->>'province' AS from_province,
    sup.address->>'zip' AS from_zip,
    sup.lat AS from_lat,
    sup.lon AS from_lon,
    f.destination->>'country' AS to_country,
    f.destination->>'province' AS to_province,
    f.destination->>'zip' AS to_zip,
    f.lat AS to_lat,
    f.lon AS to_lon,
    sup."type" AS fulfilled_by,
    COALESCE(sm.title, 'label import') AS last_mile_shipping_method,
    COALESCE(cs.integration, 'manual') AS last_mile_carrier_integration,
    COALESCE(cs.carrier_name, 'unknown') AS last_mile_carrier_name,
    cs.article_id AS last_mile_tracking_number,
    f.id::text AS fulfillment_id,
    f.status AS fulfilment_status,
    fj.id::text AS job_id,
    fj.status AS fulfilment_job_status,
    s.id::text AS shipment_id,
    s.status AS shipment_status,
    CASE 
        WHEN (DATE_PART('day', s.created_at::timestamp - f.requested_at::timestamp) - 
              (SELECT COUNT(*) 
               FROM generate_series(f.requested_at::date, s.created_at::date, '1 day') the_date 
               WHERE EXTRACT(DOW FROM the_date) IN (0, 6)) <= 1) THEN 1
        ELSE 0
    END AS within_one_business_day
FROM
    fulfillments f
    LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
    LEFT JOIN shipments s ON fj.id = s.entity_id
    LEFT JOIN merchants m ON f.merchant_id = m.id
    LEFT JOIN supply sup ON fj.supply_id = sup.id
    LEFT JOIN shipping_methods sm ON s.shipping_method_id = sm.id
    LEFT JOIN carrier_shipments cs ON cs.shipment_id = s.id::text
WHERE
    f.created_at >= '01-01-2024'
    AND f.status = 'fulfilled'
    AND fj.status != 'canceled'
    AND s.status = 'delivered'
    AND f.deleted_at IS NULL
    AND fj.deleted_at IS NULL
    AND s.deleted_at IS NULL
    AND m."name" NOT IN ('SunnyStik', 'Cinch Skincare')
    AND f."type" = 'retail'
ORDER BY
    order_requested DESC;
"""

# Execute the query and load to a DataFrame
df = pd.read_sql(query, connection)

# Ensure 'order_fulfilled' is in DATETIME format
df['order_fulfilled'] = pd.to_datetime(df['order_fulfilled'])

# Extract the month from 'order_fulfilled' and create a new column
df['month'] = df['order_fulfilled'].dt.strftime('%m')

# Group by merchant and month, and calculate the percentage
merchant_month_summary = df.groupby(['merchant', 'month']).agg(
    total_orders=pd.NamedAgg(column='within_one_business_day', aggfunc='size'),
    within_one_business_day_count=pd.NamedAgg(column='within_one_business_day', aggfunc='sum')
).reset_index()

merchant_month_summary['percentage_within_one_business_day'] = (
    merchant_month_summary['within_one_business_day_count'] / merchant_month_summary['total_orders'] * 100
)

# Format the percentage
merchant_month_summary['percentage_within_one_business_day'] = merchant_month_summary['percentage_within_one_business_day'].round(2)

# Ensure the month column is treated as a categorical type with proper ordering
merchant_month_summary['month'] = pd.Categorical(
    merchant_month_summary['month'], 
    categories=[f"{i:02}" for i in range(1, 13)], 
    ordered=True
)

# Sort the data by merchant and month
merchant_month_summary = merchant_month_summary.sort_values(['merchant', 'month'])

# Plot the data
pio.renderers.default = 'iframe'
fig = px.line(
    merchant_month_summary, 
    x='month', 
    y='percentage_within_one_business_day', 
    color='merchant', 
    labels={'month': 'Month', 'percentage_within_one_business_day': 'Percentage within 1 Business Day'},
    title='24 Business Hour Fulfillment SLA By Month'
)

fig.update_xaxes(
    title_text='Month',
    categoryorder='array',
    categoryarray=[f"{i:02}" for i in range(1, 13)]
)

fig.show()

NameError: name 'pio' is not defined

In [30]:
# Define the SQL query
_brand_order_monthly_query = """
WITH BrandMonthlyOrders AS (
    SELECT
        m."name" AS brand,
        COUNT(DISTINCT f.id) AS brand_monthly_orders,
        TO_CHAR(s.created_at, 'Month') AS month
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
        LEFT JOIN merchants m ON f.merchant_id = m.id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        m."name",
        TO_CHAR(s.created_at, 'Month')
),
MonthlyOrders AS (
    SELECT
        TO_CHAR(s.created_at, 'Month') AS month,
        COUNT(DISTINCT f.id) AS total_monthly_orders
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        TO_CHAR(s.created_at, 'Month')
),
TotalBrandOrders AS (
    SELECT
        m."name" AS brand,
        COUNT(DISTINCT f.id) AS total_brand_orders
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
        LEFT JOIN merchants m ON f.merchant_id = m.id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        m."name"
)
SELECT
    BMO.brand,
    BMO.month,
    BMO.brand_monthly_orders,
    MO.total_monthly_orders,
    TBO.total_brand_orders
FROM
    BrandMonthlyOrders BMO
    LEFT JOIN MonthlyOrders MO ON BMO.month = MO.month
    LEFT JOIN TotalBrandOrders TBO ON BMO.brand = TBO.brand
ORDER BY
    BMO.brand,
    BMO.month;
"""

# Execute the query and load data into a DataFrame
df = pd.read_sql_query(_brand_order_monthly_query, connection)

# Create a dropdown widget for brand selection
available_brands = df['brand'].unique().tolist()
brand_dropdown = widgets.Dropdown(
    options=available_brands,
    value=available_brands[0],
    description='Brand:'
)

def display_brand_orders(brand):
    # Filter data for the selected brand
    brand_data = df[df['brand'] == brand].copy()

    # Strip any extra spaces from the month column
    brand_data['month'] = brand_data['month'].str.strip()

    # Calculate the percentage of total monthly orders
    brand_data['percentage_of_total'] = (brand_data['brand_monthly_orders'] / brand_data['total_monthly_orders']) * 100

    # Format percentage to string with percentage symbol
    brand_data['percentage_of_total'] = brand_data['percentage_of_total'].apply(lambda x: f"{x:.2f}%")

    # Sort by month in ascending order
    brand_data['month'] = pd.to_datetime(brand_data['month'], format='%B').dt.strftime('%B')
    brand_data = brand_data.sort_values(by='month', ascending=True, key=lambda x: pd.to_datetime(x, format='%B'))

    # Display the table
    display(Markdown(f"### Monthly Orders for {brand}"))
    display(brand_data[['month', 'brand_monthly_orders', 'total_monthly_orders', 'percentage_of_total']])

# Use interact to create the dynamic output based on brand selection
interact(display_brand_orders, brand=brand_dropdown)


interactive(children=(Dropdown(description='Brand:', options=('Burrito Baby', 'Cinch Skincare', 'Foile Pty Ltd…

<function __main__.display_brand_orders(brand)>

In [31]:
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact

# Define the SQL query
_brand_order_monthly_revised_query = """
WITH BrandMonthlyOrders AS (
    SELECT
        m."name" AS brand,
        COUNT(DISTINCT f.id) AS brand_monthly_orders,
        TO_CHAR(s.created_at, 'Month') AS month,
        sup.address->>'country' AS country
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
        LEFT JOIN merchants m ON f.merchant_id = m.id
        LEFT JOIN supply sup ON fj.supply_id = sup.id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        m."name",
        TO_CHAR(s.created_at, 'Month'),
        sup.address->>'country'
),
MonthlyOrders AS (
    SELECT
        TO_CHAR(s.created_at, 'Month') AS month,
        COUNT(DISTINCT f.id) AS total_monthly_orders,
        sup.address->>'country' AS country
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
        LEFT JOIN supply sup ON fj.supply_id = sup.id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        TO_CHAR(s.created_at, 'Month'),
        sup.address->>'country'
),
TotalBrandOrders AS (
    SELECT
        m."name" AS brand,
        COUNT(DISTINCT f.id) AS total_brand_orders
    FROM
        fulfillments f
        LEFT JOIN line_items l ON l.entity_id = f.id
        LEFT JOIN fulfillment_jobs fj ON f.id = fj.fulfillment_id
        LEFT JOIN shipments s ON fj.id = s.entity_id
        LEFT JOIN merchants m ON f.merchant_id = m.id
    WHERE
        l.entity_type = 'fulfillment'
        AND f.created_at > '2024-01-01'
        AND f.deleted_at IS NULL
        AND fj.deleted_at IS NULL
        AND s.deleted_at IS NULL
        AND s.status = 'delivered'
    GROUP BY
        m."name"
)
SELECT
    BMO.brand,
    BMO.month,
    BMO.brand_monthly_orders,
    MO.total_monthly_orders,
    BMO.country,
    TBO.total_brand_orders
FROM
    BrandMonthlyOrders BMO
    LEFT JOIN MonthlyOrders MO ON BMO.month = MO.month AND BMO.country = MO.country
    LEFT JOIN TotalBrandOrders TBO ON BMO.brand = TBO.brand
ORDER BY
    TO_DATE(BMO.month, 'Month') DESC;
"""

# Execute the query and load data into a DataFrame
df = pd.read_sql_query(_brand_order_monthly_revised_query, connection)

# Strip any extra spaces from the month column
df['month'] = df['month'].str.strip()

# Convert the 'month' column to datetime format for sorting
df['month'] = pd.to_datetime(df['month'], format='%B').dt.strftime('%B')
df = df.sort_values(by='month', ascending=False, key=lambda x: pd.to_datetime(x, format='%B'))

# Add a geo dropdown widget
geo_dropdown = widgets.Dropdown(
    options=['All countries', 'Australia', 'United States'],
    value='All countries',
    description='Geo:'
)

# Add a brand dropdown widget
available_brands = ['All brands'] + df['brand'].unique().tolist()
brand_dropdown = widgets.Dropdown(
    options=available_brands,
    value='All brands',
    description='Brand:'
)

def update_graph(brand, geo):
    if geo == 'All countries':
        filtered_df = df
    else:
        filtered_df = df[df['country'] == geo]
    
    if brand != 'All brands':
        filtered_df = filtered_df[filtered_df['brand'] == brand]
    
    fig = px.bar(filtered_df, x='month', y='brand_monthly_orders', color='brand',
                 title=f'Monthly Orders by {brand} in {geo}',
                 labels={'brand_monthly_orders': 'Orders', 'month': 'Month'},
                 barmode='stack',
                 category_orders={'month': df['month'].unique()})
    fig.update_xaxes(title_text='Month', tickformat='%B')
    fig.update_yaxes(title_text='Orders')
    fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray': df['month'].unique()[::-1]})
    fig.show()

# Use interact to create the dynamic output based on brand and geo selection
interact(update_graph, brand=brand_dropdown, geo=geo_dropdown)


interactive(children=(Dropdown(description='Brand:', options=('All brands', 'Foile Pty Ltd', "Let's Get Explic…

<function __main__.update_graph(brand, geo)>